To do

Check Time Series data

Give a 2 page report to Sahameh about visualisation and indicate the data which has any problems, which is in consistant with the data from other dates
Planting date for 2018 is given in master thesis
Planting date for 2019 is May 05
Planting date for 2020 and remaining data will be given by Sahameh in the week starrting from 16th November

Must do normalisation of data before training

Next step is Global Mix modelling
Sahameh will share a paper from 2020 about using hyper spectral imaging, which will be basis of Mix Modelling

# Deep Learning to predict Wheat Yield

In [12]:
import os
import pandas as pd
import numpy as np

# Define the directory with data files
path = r"C:\Users\fahad\Documents\Master Thesis\Phenotyping\Data_from_2017_and_2018"
path = path.replace("\\", "/") + '/'

In [13]:
list_dir = os.listdir(path)
list_dir

['.ipynb_checkpoints',
 'Deep Learning-keras.ipynb',
 'Graminor_2018_complete.csv',
 'Masbasis_2017.xlsx',
 'Masbasis_2018_yp.xlsx',
 'Robot_2017.xlsx',
 'Robot_2018_YP.xlsx']

In [14]:
graminor_18 = list_dir[2]
masbasis_17 = list_dir[3]
masbasis_18 = list_dir[4]
robot_17 = list_dir[5]
robot_18 = list_dir[6]

data_file_list = [graminor_18, masbasis_17, masbasis_18, robot_17, robot_18]

In [15]:
# Get the name of the variable as string

import inspect

def retrieve_name(var):
        """
        Gets the name of var. Does it from the out most frame inner-wards.
        :param var: variable to get name from.
        :return: string
        """
        for fi in reversed(inspect.stack()):
            names = [var_name for var_name, var_val in fi.frame.f_locals.items() if var_val is var]
            if len(names) > 0:
                return names[0]


In [16]:
from IPython.display import display, HTML

df_str_list = []

for filename in data_file_list:
    new_df = retrieve_name(filename) + '_df'
    if filename[-3:] == 'csv':
        locals()[new_df] = pd.read_csv(path + filename,
                                       delimiter=';',
                                       encoding="ISO-8859-1")
        df_str_list.append(new_df)
        # Encoding has been defined to avoid UnicodeDecodeError while reading the csv
#         display(locals()[new_df].head(10))    # Print the df in pretty format
    elif filename[-4:] == 'xlsx':
        locals()[new_df] = pd.ExcelFile(path + filename)
        if len(locals()[new_df].sheet_names) == 1:
            locals()[new_df] = pd.read_excel(path + filename)
            df_str_list.append(new_df)
        elif len(locals()[new_df].sheet_names) > 1:
            for i in locals()[new_df].sheet_names:
                # Replacing '.' and '-' in the sheet file name to '_'
                k = i.replace('.', '_')
                k = k.replace('-', '_')
                new_sheet_df = retrieve_name(filename) + '_' + k + '_df'
                locals()[new_sheet_df] = pd.read_excel(path + filename, sheet_name = i)
                df_str_list.append(new_sheet_df)

#         display(locals()[new_df].head(10))    # Print the df in pretty format

print(*df_str_list, sep = "\n")

graminor_18_df
masbasis_17_df
masbasis_18_df
robot_17_14_06_17_df
robot_17_19_06_17_df
robot_17_29_06_17_df
robot_17_03_07_17_df
robot_17_14_07_17_df
robot_17_17_07_17_df
robot_17_14_08_17_df
robot_18_02_07_18_df
robot_18_smallfield05_07_18_df
robot_18_smallfield11_07_18_df
robot_18_smallfield19_07_18_df
robot_18_smallfield24_07_18_df


In [17]:
lists_of_median_cols = []

for df in df_str_list:
    list_of_columns = df+'_median_columns'
    locals()[list_of_columns] = []
    for strg in locals()[df].columns.tolist():
        if strg.find('median') != -1 or strg.find('mtci') != -1 or strg.find('evi') != -1 or strg.find('MAT') != -1:
            locals()[list_of_columns].append(strg)
    lists_of_median_cols.append(list_of_columns)
    
    print('\n', list_of_columns,locals() [df].shape, *locals()[list_of_columns], sep='\n')



graminor_18_df_median_columns
(599, 109)
26-06-18_bluemedian
26-06-18_greenmedian
26-06-18_ndvimedian
26-06-18_nirmedian
26-06-18_redmedian
26-06-18_rededgemedian
26-06-18_mtci
MAT
26-06-18_evi
02-07-18_bluemedian
02-07-18_greenmedian
02-07-18_ndvimedian
02-07-18_nirmedian
02-07-18_redmedian
02-07-18_rededgemedian
02-07-18_mtci
02-07-18MAT
02-07-18_evi
19-07-18_bluemedian
19-07-18_greenmedian
19-07-18_ndvimedian
19-07-18_nirmedian
19-07-18_redmedian
19-07-18_rededgemedian
19-07-18_mtci
19-07-18MAT
19-07-18_evi
02-07-18_mtci - 26-06-18_mtci
19-07-18_mtci - 26-06-18_mtci


masbasis_17_df_median_columns
(560, 179)
14-07-17_greenmedian
14-07-17_ndvimedian
14-07-17_nirmedian
14-07-17_redmedian
14-07-17_rededgemedian
14-07-17_mtci
MAT
17-07-17_greenmedian
17-07-17_ndvimedian
17-07-17_nirmedian
17-07-17_redmedian
17-07-17_rededgemedian
17-07-17_mtci
17-07-17MAT
20-07-17_greenmedian
20-07-17_ndvimedian
20-07-17_nirmedian
20-07-17_redmedian
20-07-17_rededgemedian
20-07-17_mtci
20-07-17MAT
01-

In [18]:
lists_of_yield_cols = []

for df in df_str_list:
    list_of_columns = df+'_yield_columns'
    locals()[list_of_columns] = []
    for strg in locals()[df].columns.tolist():
        if strg.find('Yield') != -1:
            locals()[list_of_columns].append(strg)
    lists_of_yield_cols.append(list_of_columns)
    
    print('\n', list_of_columns,locals() [df].shape, *locals()[list_of_columns], sep='\n')



graminor_18_df_yield_columns
(599, 109)
26-06-18GrainYield
02-07-18GrainYield
19-07-18GrainYield


masbasis_17_df_yield_columns
(560, 179)
14-07-17Yield
17-07-17Yield
20-07-17Yield
01-08-17Yield


masbasis_18_df_yield_columns
(528, 101)
13-07-18GrainYield
26-07-18GrainYield


robot_17_14_06_17_df_yield_columns
(96, 33)
GrainYield
HarvestYield 


robot_17_19_06_17_df_yield_columns
(96, 73)
GrainYield
HarvestYield 


robot_17_29_06_17_df_yield_columns
(96, 73)
GrainYield
HarvestYield 


robot_17_03_07_17_df_yield_columns
(96, 73)
GrainYield
HarvestYield 


robot_17_14_07_17_df_yield_columns
(96, 73)
GrainYield
HarvestYield 


robot_17_17_07_17_df_yield_columns
(96, 73)
GrainYield
HarvestYield 


robot_17_14_08_17_df_yield_columns
(96, 73)
GrainYield
HarvestYield 


robot_18_02_07_18_df_yield_columns
(96, 22)
GrainYield


robot_18_smallfield05_07_18_df_yield_columns
(96, 22)
GrainYield


robot_18_smallfield11_07_18_df_yield_columns
(96, 22)
GrainYield


robot_18_smallfield19_07_18_df_yi

## Machine Learning Part

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [20]:
robot_18_smallfield24_07_18_df[robot_18_smallfield24_07_18_df_yield_columns]

,GrainYield
0,195.033333
1,223.100000
2,217.483333
3,188.750000
4,155.716667
...,...
91,283.083333
92,269.700000
93,278.150000
94,337.233333


In [21]:
#==============================================================================
# Split dataframe into data and target
#==============================================================================

X = robot_18_smallfield24_07_18_df[
    robot_18_smallfield24_07_18_df_median_columns]
y = robot_18_smallfield24_07_18_df[
    robot_18_smallfield24_07_18_df_yield_columns]

In [22]:
#==============================================================================
# Create separate train/test splits from Main data
#==============================================================================
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=55)

In [23]:
#==============================================================================
# Scale features using StandardScaler class in scikit-learn
#==============================================================================

# Initialise standard scaler and compute mean and STD from training data
sc = StandardScaler()
sc.fit(X_train)

# Transform (standardise) both X_train and X_test with mean and STD from
# training data
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [24]:
#==============================================================================
# Defining the function to vaiidate the model with the test data and
# get the results from regression evaluation metrices in sklearn
#==============================================================================
pred = []
accuracy = []


def test_data_regression(model, X_test, y_test):
    pred = []
    accuracy = []
    #==============================================================================
    # Make predictions for test set
    #==============================================================================

    # Predict classes for samples in test set
    y_pred = model.predict(X_test)

    #==============================================================================
    # Compute performance
    #==============================================================================

    from sklearn.metrics import mean_absolute_error
    mae = mean_absolute_error(y_test, y_pred)
    print(mae, ' mean_absolute_error')
    accuracy.append(mae)

    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(y_test, y_pred, squared=True)
    print(mse, ' mean_squared_error')
    accuracy.append(mse)

    from sklearn.metrics import mean_squared_error
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print(rmse, ' root_mean_squared_error')
    accuracy.append(rmse)

    from sklearn.metrics import r2_score
    r2 = r2_score(y_test, y_pred)
    print(r2, ' r2_score')
    accuracy.append(r2)

    # Print accuracy computed from predictions on the test set
    print(accuracy)

    #==============================================================================
    # Append Results
    #==============================================================================
    results = []
    import datetime
    datetime = datetime.datetime.now()
    results.append((model, 'MAE = {}'.format(mae), 'MSE = {}'.format(mse),
                    'RMSE = {}'.format(rmse), 'R2 = {}'.format(r2),
                    'List = {}'.format(accuracy), datetime))

    pd.DataFrame(np.asarray(results)).to_csv('results.csv',
                                             mode='a',
                                             header=None)
    pred.extend(y_pred)

In [25]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=1000,
                              max_depth=250,
                              min_samples_split=5,
                              random_state=0,
                              n_jobs=-1)
model.fit(X_train_std, y_train)
y_pred = model.predict(X_test_std)
test_data_regression(model, X_test_std, y_test)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


31.791968968083356  mean_absolute_error
1510.5850790239026  mean_squared_error
38.86624601146736  root_mean_squared_error
0.476003836137754  r2_score
[31.791968968083356, 1510.5850790239026, 38.86624601146736, 0.476003836137754]


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [26]:
from sklearn.cross_decomposition import PLSRegression

model = PLSRegression(n_components=5)
model.fit(X_train_std, y_train)
y_pred = model.predict(X_test_std)
test_data_regression(model, X_test_std, y_test)

26.069610149758933  mean_absolute_error
1146.6873396491621  mean_squared_error
33.862772179034046  root_mean_squared_error
0.6022337467322105  r2_score
[26.069610149758933, 1146.6873396491621, 33.862772179034046, 0.6022337467322105]


In [27]:
y_pred

array([[228.43445511],
       [208.16495241],
       [229.30338632],
       [146.42192367],
       [175.99016696],
       [265.87055526],
       [182.72079741],
       [150.79555018],
       [179.94328825],
       [177.0734073 ],
       [172.45244835],
       [253.03559614],
       [197.43453425],
       [222.45180587],
       [181.9630764 ],
       [176.08251618],
       [193.21170533],
       [208.14538433],
       [178.39723128],
       [225.7900946 ],
       [189.53237185],
       [183.57667404],
       [154.35145008],
       [213.56089343],
       [139.02729324],
       [191.62636537],
       [144.79200749],
       [190.33373498],
       [209.64883211]])

In [28]:
import numpy as np
import h5py
import pandas as pd

# Visualization
import cv2
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt

# Preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K

# Neural networks
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model

# Training
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Define eval metric and loss function (DICE)

In [31]:
"""
Metric used in the competition
""" 
# Defining the dice_coef function
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Defining the dice_loss function
def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

# Create U-net architecture

In [32]:
"""
Version of U-Net with dropout and size preservation (padding= 'same')
""" 
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x


def get_deep_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True, n_classes = 1, growth_factor=2, upconv=True):

    inputs = input_img
    #inputs = BatchNormalization()(inputs)
    
    # Creating deep convnets using Conv2D and MaxPooling2D in each convolutional layer
    conv1 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    #pool1 = Dropout(droprate)(pool1)

    n_filters *= growth_factor
    # Applying batch normalization to each layer to be able to use deep convnets
    pool1 = BatchNormalization()(pool1)
    # Second Covnet
    conv2 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(dropout)(pool2)

    n_filters *= growth_factor
    # Applying batch normalization
    pool2 = BatchNormalization()(pool2)
    # Third Covnet
    conv3 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(dropout)(pool3)

    n_filters *= growth_factor
    # Applying batch normalization
    pool3 = BatchNormalization()(pool3)
    conv4_0 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(pool3)
    conv4_0 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv4_0)
    pool4_1 = MaxPooling2D(pool_size=(2, 2))(conv4_0)
    pool4_1 = Dropout(dropout)(pool4_1)

    n_filters *= growth_factor
    # Applying batch normalization
    pool4_1 = BatchNormalization()(pool4_1)
    conv4_1 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(pool4_1)
    conv4_1 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv4_1)
    pool4_2 = MaxPooling2D(pool_size=(2, 2))(conv4_1)
    pool4_2 = Dropout(dropout)(pool4_2)

    n_filters *= growth_factor
    conv5 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(pool4_2)
    conv5 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv5)

    n_filters //= growth_factor
    # Using image upsampling by either Conv2DTranspose or UpSampling2D
    if upconv:
        up6_1 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv5), conv4_1])
    else:
        up6_1 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4_1])
    # Applying batch normalization
    up6_1 = BatchNormalization()(up6_1)
    conv6_1 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(up6_1)
    conv6_1 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv6_1)
    conv6_1 = Dropout(dropout)(conv6_1)

    # Using image upsampling by either Conv2DTranspose or UpSampling2D
    n_filters //= growth_factor
    if upconv:
        up6_2 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv6_1), conv4_0])
    else:
        up6_2 = concatenate([UpSampling2D(size=(2, 2))(conv6_1), conv4_0])
    up6_2 = BatchNormalization()(up6_2)
    conv6_2 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(up6_2)
    conv6_2 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv6_2)
    conv6_2 = Dropout(dropout)(conv6_2)

    # Using image upsampling by either Conv2DTranspose or UpSampling2D
    n_filters //= growth_factor
    if upconv:
        up7 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv6_2), conv3])
    else:
        up7 = concatenate([UpSampling2D(size=(2, 2))(conv6_2), conv3])
    # Applying batch normalization
    up7 = BatchNormalization()(up7)
    conv7 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = Dropout(dropout)(conv7)

    # Using image upsampling by either Conv2DTranspose or UpSampling2D
    n_filters //= growth_factor
    if upconv:
        up8 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv7), conv2])
    else:
        up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2])
    # Applying batch normalization
    up8 = BatchNormalization()(up8)
    conv8 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = Dropout(dropout)(conv8)

    # Using image upsampling by either Conv2DTranspose or UpSampling2D
    n_filters //= growth_factor
    if upconv:
        up9 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv8), conv1])
    else:
        up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1])
    conv9 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(n_classes, (1, 1), activation='sigmoid')(conv9)
    
    # Creating model and feeding the input image and the final convolutional layer into it
    model = Model(inputs=inputs, outputs=conv10)

    return model

In [33]:
"""
Data augmentation for training dataset
"""

# Define batch size for augmentation
batch_size = 64
# Define random seed
seed = 123

# Dict containing augmentation variables
data_gen_args = dict(
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    rotation_range=90,
    vertical_flip=True,
    horizontal_flip=True,
)

# Instantiate DataGenerator class with variables from dict
train_datagen = ImageDataGenerator(**data_gen_args)

# Fit DataGen instance to X_train (Only needed for standardization etc.)
train_datagen.fit(X_train)

# Create flow for training images
image_train_generator = train_datagen.flow(
    X_train,
    None,
    seed=seed,
    batch_size=batch_size,
)

# Create separate flow for training masks
mask_train_generator = train_datagen.flow(
    y_train,
    None,
    seed=seed,
    batch_size=batch_size,
)

# Combine generators into one which yields image and masks
train_generator = zip(image_train_generator, mask_train_generator)
"""
Model creation and compilation
"""

# Define input class with correct shape
input_img = Input(shape=(128, 128, 4))

# Number of filters in the U-net
n_filters = 64

# Instantiate U-net without dropout with batchnorm
model = get_deep_unet(input_img,
                      n_filters=n_filters,
                      dropout=0.0,
                      batchnorm=True,
                      n_classes=1)

# Compile model using "Adam" with custom loss function and metrics
model.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coef])
"""
Callbacks
"""

# Early stopping for saving compute time
earlystopping = EarlyStopping(monitor='val_dice_coef',
                              verbose=1,
                              min_delta=0.01,
                              patience=3,
                              mode='max')

# Saving best models
model_path = '/content/drive/My Drive/Skole/CA2/tmp/unet.model'
checkpoint = ModelCheckpoint(model_path,
                             monitor='val_loss',
                             save_best_only=True,
                             verbose=1,
                             mode='min')

# Reduce learning rate to improve accuracy on plateaus
redlr = ReduceLROnPlateau(factor=0.1,
                          patience=1,
                          min_lr=1e-08,
                          verbose=1,
                          monitor='val_loss',
                          mode='min'),

# Collect callbacks in list
callbacks_list = [redlr, earlystopping, checkpoint]
"""
Model training
"""

history = model.fit(train_generator,
                    steps_per_epoch=(X_train.shape[0] // n_filters),
                    validation_data=(X_val, y_val),
                    validation_steps=(X_val.shape[0] // n_filters),
                    epochs=4,
                    verbose=1,
                    callbacks=callbacks_list)

ValueError: Input to `.fit()` should have rank 4. Got array with shape: (67, 8)